## COAL INFRA MODEL

In [6]:
import Pkg; Pkg.add("Plots")

    Updating registry at `C:\Users\charl\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\charl\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\charl\.julia\environments\v1.11\Manifest.toml`


In [7]:
using Plots
using Pkg
using DataFrames, CSV
using CSV
using JuMP
using HiGHS
using Statistics

### DATA

In [ ]:
generators = DataFrame(CSV.File("data/Generators_data.csv"))
generators = select(generators, :R_ID, :Resource, :zone, :THERM, :DISP, :NDISP, :STOR, :HYDRO, :RPS, :CES,
                    :Commit, :Existing_Cap_MW, :Existing_Cap_MWh, :Cap_size, :New_Build, :Max_Cap_MW,
                    :Inv_cost_per_MWyr, :Fixed_OM_cost_per_MWyr, :Inv_cost_per_MWhyr, :Fixed_OM_cost_per_MWhyr,
                    :Var_OM_cost_per_MWh, :Start_cost_per_MW, :Start_fuel_MMBTU_per_MW, :Heat_rate_MMBTU_per_MWh, :Fuel,
                    :Min_power, :Ramp_Up_percentage, :Ramp_Dn_percentage, :Up_time, :Down_time,
                    :Eff_up, :Eff_down); 
G = generators.R_ID # set of gens 

UndefVarError: UndefVarError: `DataFrame` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [19]:
demand_inputs = DataFrame(CSV.File("data/Load_data.csv"))
VOLL = demand_inputs.Voll[1]
S = convert(Array{Int64}, collect(skipmissing(demand_inputs.Demand_segment)))

nse = DataFrame(Segment=S, 
                NSE_Cost = VOLL.*collect(skipmissing(demand_inputs.Cost_of_demand_curtailment_perMW)),
                NSE_Max = collect(skipmissing(demand_inputs.Max_demand_curtailment)))

hours_per_period = convert(Int64, demand_inputs.Hours_per_period[1])
P = convert(Array{Int64}, 1:demand_inputs.Subperiods[1])
W = convert(Array{Int64}, collect(skipmissing(demand_inputs.Sub_Weights)))
T = convert(Array{Int64}, demand_inputs.Time_index)
sample_weight = zeros(Float64, size(T,1))
t=1

for p in P
    for h in 1:hours_per_period
        sample_weight[t] = W[p]/hours_per_period
        t=t+1
    end
end

Z = convert(Array{Int64}, 1:3)
demand = select(demand_inputs, :Load_MW_z1, :Load_MW_z2, :Load_MW_z3);

In [24]:
variability = DataFrame(CSV.File("data/Generators_variability.csv"))
variability = variability[:,2:ncol(variability)];
fuels = DataFrame(CSV.File("data/Fuels_data.csv"));

In [ ]:
generators.Var_Cost = zeros(Float64, size(G,1))
generators.CO2_Rate = zeros(Float64, size(G,1))
generators.Start_Cost = zeros(Float64, size(G,1))
generators.CO2_Per_Start = zeros(Float64, size(G,1))

for g in G
    generators.Var_Cost[g] = generators.Var_OM_cost_per_MWh[g] + fuels[fuels.Fuel.==generators.Fuel[g],:Cost_per_MMBtu][1]*generators.Heat_rate_MMBTU_per_MWh[g]
    generators.CO2_Rate[g] = fuels[fuels.Fuel.==generators.Fuel[g],:CO2_content_tons_per_MMBtu][1]*generators.Heat_rate_MMBTU_per_MWh[g]
    generators.Start_Cost[g] = generators.Start_cost_per_MW[g] + fuels[fuels.Fuel.==generators.Fuel[g],:Cost_per_MMBtu][1]*generators.Start_fuel_MMBTU_per_MW[g]
    generators.CO2_Per_Start[g] = fuels[fuels.Fuel.==generators.Fuel[g],:CO2_content_tons_per_MMBtu][1]*generators.Start_fuel_MMBTU_per_MW[g]
end

# JULIA Model 1

### Sets

- $Y$: Years  {1, n}
- $T$: Time within a year (optional granularity, e.g. hours/seasons)  {1...8760}
- *$L$: Locations of coal power plants (Maybe?)*
- $G$: Generator technologies  
  - Subsets: existing (`EXIST`), new (`NEW`), repurposed (`REPURPOSED`)


### Parameters

- $DEMAND_{t,y}$ : Demand in year $y$, time $t$  

- $INV_g$ : Investment cost for tech $g$ [\$/MW]  

- $FoM_g$ : Fixed O&M [\$/MW-yr]  

- $LIFE_g$ : Lifetime of tech $g$ [years]  

- $LEAD_g$ : Lead time before new build becomes available [years]  

- $VC_g$ : Variable generation cost [\$/MWh]  

- $CF_{g,t}$ : Capacity factor of tech $g$ at time $t$  

- $Fp_g$ : Footprint coefficient of tech $g$ (e.g. land constraint)  

- $A_g$ : Area limit (location-specific, e.g. for renewables)  

- $PLANT_g$ : Existing capacity [MW]  

- $EF_g$ : Emission factor [tCO$_$/MWh]  

- $PCO2_y$ : Carbon price in year $y$ [\$/tCO$_$]  

- $VOLL_{t,y}$ : Value of Lost Load [\$/MWh]  

- $r$ : Discount rate


### Decision Variables

- $use_{t,y} \geq 0$ : Unserved energy demand at time $t, y$

- $gen_{g,t,y} \geq 0$ : Electricity generation from tech $g$  

- $build_{g,y} \geq 0$ : New build capacity for tech $g$  

- $ret_{g,y} \geq 0$ : Retired capacity for tech $g$  

- $cap_{g,y} \geq 0$ : Total available capacity of tech $g$


### Objective Function

$$
\min_{g,y,t} \;
\Bigg[
\sum_{g,y,t} \frac{1}{(1+r)^y} \, (VC_g + EF_g \cdot PCO2_y) \cdot gen_{g,t,y}
+ \sum_{g,y} \frac{1}{(1+r)^y} \, FoM_g \cdot cap_{g,y}
+ \sum_{g,y} \frac{1}{(1+r)^y} \, INV_g \cdot build_{g,y}
+ \sum_{y,t} \frac{1}{(1+r)^y} \, VOLL_{t,y} \cdot use_{t,y}
\Bigg]
$$


### Constraints

**Demand balance (with demand-side flexibility)**  
$$
\sum_g gen_{g,t,y} + use_{t,y} = DEMAND_{t,y}, \quad \forall t,y
$$

**Generation limited by capacity & capacity factor**  
$$
gen_{g,t,y} \leq cap_{g,y} \cdot CF_{g,t}, \quad \forall g,t,y
$$

**Initial capacity**  
$$
cap_{g,1} = PLANT_g, \quad \forall g
$$

**Capacity evolution (with lead time and retirement)**  
$$
cap_{g,y} =
\begin{cases}
PLANT_g, & y = 1 \\[6pt]
cap_{g,y-1} - ret_{g,y}, & 2 \leq y \leq LEAD_g \\[6pt]
cap_{g,y-1} + build_{g,y-LEAD_g} - ret_{g,y}, & y > LEAD_g
\end{cases}
$$

**Lifetime constraint**  # Remove don't need to retire in the horizon 
$$
ret_{g,y} \geq build_{g,y-LIFE_g}, \quad \forall g,y
$$

**Land/footprint constraint**  ## next step
$$
\sum_g Fp_g \cdot build_{g,y} \leq A_g, \quad \forall y
$$



### Julia JuMP implementation 


In [25]:
#sets
y0 = 0
Y  = y0:25 # years
T  = 1:8760                         
G  = [:coal, :gas, :solar, :wind, :biomass]

5-element Vector{Symbol}:
 :coal
 :gas
 :solar
 :wind
 :biomass

In [29]:
#parameters (Made Up - Change)

r = Dict(
    :coal    => 0.04,   
    :biomass => 0.048,
    :solar   => 0.038,
    :wind    => 0.03,
    :gas     => 0.035
)

# Demand (MWh) per slice/year
Demand_Data = CSV.read("data/demand_25yrs_forecast.csv", DataFrame)
DEMAND = Dict((row.Year ,row.Hour) => row.Demand for row in eachrow(Demand_Data))

# Existing capacity (MW) in base year
PLANT = Dict(:coal=>4000.0, :gas=>2000.0, :solar=>800.0, :wind=>1200.0, :biomass=>0.0)

# Costs
INV = Dict(
    :coal    => 999_999_999.0,   # placeholder
    :gas     => 1_000_000.0,   # CCGT greenfield
    :biomass => 5_391_000.0,   # Biomass greenfield
    :solar   =>   750_000.0,
    :wind    => 1_000_000.0
) # $/MW

FoM = Dict(
    :coal    => 120_000.0,
    :gas     => 28_000.0,    # CCGT
    :biomass => 157_000.0,   # biomass greenfield
    :solar   => 15_000.0,
    :wind    => 40_000.0
) # $/MW-yr

VC = Dict(
    :coal    => 24.2,
    :gas     => 27.6,     # CCGT
    :biomass => 76.0,     # 5 VOM + 71 fuel
    :solar   => 0.0,
    :wind    => 0.0
) # $/MWh
EF = Dict(
    :coal    => 0.774,
    :biomass => 1.502,   # or 0.0 if carbon-neutral assumption
    :gas     => 0.340,
    :solar   => 0.0,
    :wind    => 0.0
)# tCO₂/MWh

# Carbon price path
PCO2 = Dict(y => 90.44 for y in Y)                                                       # $/tCO₂

# Value of lost load
VOLL = Dict((y,t) => 10_000.0 for y in Y, t in T)                                        # $/MWh

# Capacity factors
CF_Data = CSV.read("data/CFData_Expanded.csv", DataFrame)

CF = Dict()
for row in eachrow(CF_Data)
    h = row.Hour
    CF[(:coal, h)]    = row.Coal
    CF[(:gas, h)]     = row.Gas
    CF[(:biomass, h)] = row.Biomass
    CF[(:wind, h)]    = row.Wind
    CF[(:solar, h)]   = row.Solar
end

# Lifetime and lead time (years)
LIFE = Dict(:coal=>30, :gas=>30, :solar=>30, :wind=>30, :biomass=>30) 
LEAD = Dict(:coal=>5,  :gas=>3,  :solar=>1,  :wind=>1,  :biomass=>2)


Dict{Symbol, Int64} with 5 entries:
  :wind    => 1
  :coal    => 5
  :gas     => 3
  :solar   => 1
  :biomass => 2

In [30]:
model = Model(HiGHS.Optimizer)

@variable(model, gen[g in G, t in T, y in Y] >= 0)    # generation [MWh]
@variable(model, cap[g in G, y in Y] >= 0)            # available capacity [MW]
@variable(model, build[g in G, y in Y] >= 0)          # new builds [MW]
@variable(model, retire[g in G, y in Y] >= 0)         # retirements [MW]
@variable(model, use[t in T, y in Y] >= 0)           # unserved load [MWh]

2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, 1:8760
    Dimension 2, 0:25
And data, a 8760×26 Matrix{VariableRef}:
 use[1,0]     use[1,1]     use[1,2]     …  use[1,24]     use[1,25]
 use[2,0]     use[2,1]     use[2,2]        use[2,24]     use[2,25]
 use[3,0]     use[3,1]     use[3,2]        use[3,24]     use[3,25]
 use[4,0]     use[4,1]     use[4,2]        use[4,24]     use[4,25]
 use[5,0]     use[5,1]     use[5,2]        use[5,24]     use[5,25]
 use[6,0]     use[6,1]     use[6,2]     …  use[6,24]     use[6,25]
 use[7,0]     use[7,1]     use[7,2]        use[7,24]     use[7,25]
 use[8,0]     use[8,1]     use[8,2]        use[8,24]     use[8,25]
 use[9,0]     use[9,1]     use[9,2]        use[9,24]     use[9,25]
 use[10,0]    use[10,1]    use[10,2]       use[10,24]    use[10,25]
 ⋮                                      ⋱                ⋮
 use[8752,0]  use[8752,1]  use[8752,2]     use[8752,24]  use[8752,25]
 use[8753,0]  use[8753,1]  use[8753,2]     use[8

In [ ]:
@objective(model, Min,
    # Variable generation cost + carbon
    sum((VC[g] + EF[g]*PCO2[y]) * gen[g,t,y] / (1+r[g])^(y-y0) for g in G, t in T, y in Y) +
    # Fixed O&M
    sum(FoM[g] * cap[g,y] / (1+r[g])^(y-y0) for g in G, y in Y) +
    # Investment
    sum(INV[g] * build[g,y] / (1+r[g])^(y-y0) for g in G, y in Y) +
    # Unserved demand (VOLL penalty)
    sum(VOLL[(y,t)] * use[t,y] / (1+0.05)^(y-y0) for t in T, y in Y) # set r to 0.05 here
)

94.20056 gen[coal,1,0] + 90.57746153846153 gen[coal,1,1] + 87.09371301775147 gen[coal,1,2] + 83.74395482476103 gen[coal,1,3] + 80.52303348534713 gen[coal,1,4] + 77.42599373591071 gen[coal,1,5] + 74.44807089991413 gen[coal,1,6] + 71.58468355760975 gen[coal,1,7] + 68.83142649770168 gen[coal,1,8] + 66.18406394009776 gen[coal,1,9] + 63.63852301932477 gen[coal,1,10] + 61.190887518581505 gen[coal,1,11] + 58.837391844789906 gen[coal,1,12] + 56.57441523537491 gen[coal,1,13] + 54.39847618786049 gen[coal,1,14] + 52.30622710371201 gen[coal,1,15] + 50.294449138184625 gen[coal,1,16] + 48.36004724825444 gen[coal,1,17] + 46.50004543101388 gen[coal,1,18] + 44.711582145205654 gen[coal,1,19] + 42.99190590885159 gen[coal,1,20] + 41.33837106620345 gen[coal,1,21] + 39.74843371750332 gen[coal,1,22] + 38.21964780529165 gen[coal,1,23] + 36.74966135124197 gen[coal,1,24] + 35.33621283773267 gen[coal,1,25] + 94.20056 gen[coal,2,0] + 90.57746153846153 gen[coal,2,1] + 87.09371301775147 gen[coal,2,2] + 83.743954824

In [32]:
# Demand balance
@constraint(model, [y in Y, t in T],
    sum(gen[g,t,y] for g in G) + use[t,y] == DEMAND[(y,t)]
)

# Generation limited by capacity × CF
@constraint(model, [g in G, y in Y, t in T],
    gen[g,t,y] <= cap[g,y] * CF[(g,t)]
)

# Initial capacity
@constraint(model, [g in G],
    cap[g,y0] == PLANT[g]
)

# Capacity dynamics (with lead times)
for g in G, (i,y) in enumerate(Y)
    if y > y0
        if y - y0 >= LEAD[g]
            @constraint(model, cap[g,y] == cap[g,y-1] + build[g,y-LEAD[g]] - retire[g,y])
        else
            @constraint(model, cap[g,y] == cap[g,y-1] - retire[g,y])
        end
    end
end

# Lifetime constraint
for g in G, y in Y
    if y - y0 >= LIFE[g]
        @constraint(model, retire[g,y] >= build[g,y-LIFE[g]])
    end
end


In [ ]:
optimize!(model)

println("Objective value: ", objective_value(model))

In [17]:
cap_df = DataFrame(
    tech = repeat(G, inner=length(Y)),
    year = vcat([fill(y, length(G)) for y in Y]...),
    MW   = [value(cap[g,y]) for y in Y for g in G]
)

build_df = DataFrame(
    tech = repeat(G, inner=length(Y)),
    year = vcat([fill(y, length(G)) for y in Y]...),
    MW   = [value(build[g,y]) for y in Y for g in G]
)

retire_df = DataFrame(
    tech = repeat(G, inner=length(Y)),
    year = vcat([fill(y, length(G)) for y in Y]...),
    MW   = [value(retire[g,y]) for y in Y for g in G]
)


Row,tech,year,MW
,Symbol,Int64,Float64
1,coal,0,0.0
2,coal,0,0.0
3,coal,0,0.0
4,coal,0,0.0
5,coal,0,0.0
6,coal,1,0.0
7,coal,1,0.0
8,coal,1,0.0
9,coal,1,0.0


In [18]:
cap_df

Row,tech,year,MW
,Symbol,Int64,Float64
1,coal,0,4000.0
2,coal,0,2000.0
3,coal,0,800.0
4,coal,0,1200.0
5,coal,0,-0.0
6,coal,1,4000.0
7,coal,1,2000.0
8,coal,1,44845.6
9,coal,1,9.85211e5
